In [2]:
import numpy as np
from data_loader.data_loaders import NinaPro_Dataset

train = NinaPro_Dataset(DB=1, subject=10, trainRepeatIndex=[0,1,3,4,6,8,9,10])
tests = NinaPro_Dataset(DB=1, subject=10, trainRepeatIndex=[2,5,7])
print('Dimension of feature space: ', train.Xs.shape)
print('Tests samples numbers: ', tests.Xs.shape)

Feature Extraction: 100%|██████████| 68/68 [00:31<00:00,  2.17it/s]
Dimension of feature space:  (2605, 7630)
Tests samples numbers:  (1005, 7630)


In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imp.fit(train.Xs)
train.Xs = imp.transform(train.Xs)
tests.Xs = imp.transform(tests.Xs)
clf = RandomForestClassifier(max_depth=100,n_estimators=1000)
clf.fit(train.Xs, train.Ys)
print('Train: ', clf.score(train.Xs, train.Ys))
print('Tests: ', clf.score(tests.Xs, tests.Ys))

Train:  1.0
Tests:  0.8507462686567164


In [11]:
from pycaret.classification import *
import pandas as pd

#修改，把下面features_Extracted改成自己的features(从tsfresh提出来的pandas dataframe格式)，data_Y修改为为数据标签列
features_With_Labels = pd.DataFrame(train.Xs)
features_With_Labels["labels"] = pd.DataFrame(train.Ys.squeeze(1))

# models_List = ['lr', 'knn', 'nb', 'dt', 'svm', 'rbfsvm', 'gpc', 'mlp', 'ridge','rf', 'qda', 'ada', 'gbc', 'lda', 'et', 'xgboost', 'lightgbm']
models_List = [dt','rf', 'et', 'xgboost', 'lightgbm']
# models_List = ['lr', 'nb']

setup(features_With_Labels, target = 'labels', silent= True, 
      train_size=0.9)
set_config('X_train', pd.DataFrame(train.Xs))
set_config('y_train', pd.DataFrame(train.Ys.squeeze(1)))
set_config('X_test',  pd.DataFrame(tests.Xs))
set_config('y_test',  pd.DataFrame(tests.Ys.squeeze(1)))
set_config('seed', 1)
best_specific = compare_models(whitelist = models_List, fold=10)
tuned_model = tune_model(best_specific)
predict_model(tuned_model)


Setup Succesfully Completed!


,Description,Value
0,session_id,7365
1,Target Type,Multiclass
2,Label Encoded,None
3,Original Data,"(2605, 7631)"
4,Missing Values,False
5,Numeric Features,7430
6,Categorical Features,200
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


IntProgress(value=0, description='Processing: ', max=31)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,16:42:56
Status,. . . . . . . . . . . . . . . . . .,Compiling Final Model
Estimator,. . . . . . . . . . . . . . . . . .,Logistic Regression
ETC,. . . . . . . . . . . . . . . . . .,Almost Finished


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
0,Logistic Regression,0.2476,0.0,0.2526,0.2562,0.2396,0.2327,0.2333,48.0233
1,Naive Bayes,0.2085,0.0,0.2028,0.2329,0.1745,0.1913,0.1971,19.6418


# best model for tests datasets